# Accessing and exploration of the AACT database

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import os
import requests
from sqlalchemy import create_engine

## Accessing AACT database

In [3]:
# Connect to database
aact_host=os.environ["aact_host"]
aact_port=os.environ["aact_port"]
db_name=os.environ["db_name"]
aact_username=os.environ["aact_username"]
aact_password=os.environ["aact_password"]

db_string = "postgres://{}:{}@{}:{}/{}". \
    format(aact_username, aact_password, aact_host, aact_port, db_name)

db = create_engine(db_string)

conn = db.connect()

In [4]:
# Perform count of studies
q = "select count(*) from studies;"

conn.execute(q).fetchall()

[(306660,)]

## Explore data relevant to patients
The AACT database contains ___ tables with ___ data. In order to 
We'll be extracting important information that will be important to patients. I wanted to prioritize which sections will be most important for patients for understanding whether they will be eligible for a trial. We concentrated on information such as the active participation period, the follow-up period, and eligibility criteria.



We designed the categories from our manual analysis of 100 sample trial descriptions. The categories include the information most vital to patients’ understanding of a trial: active participation period, follow up period, and eligibility criteria, to name a few.

Next we labelled each category with its datatype, and tagged each with keywords that appear in or near the information we want.

To assess the validity of the keywords we chose, we conducted frequency evaluations of each indicator. Here’s a sample of what the output looks like:

In [5]:
# Perform selection of brief_summary
condition = "carcinoma, hepatocellular"

q = """SELECT studies.nct_id,
            e.gender,
            e.minimum_age,
            e.maximum_age,
            e.healthy_volunteers,
            e.population,
            e.criteria AS eligibility_criteria,
            bs.description AS brief_summary,
            
            c.downcase_name AS condition
        FROM studies
        FULL OUTER JOIN brief_summaries bs
            ON studies.nct_id=bs.nct_id
        FULL OUTER JOIN eligibilities e
            ON studies.nct_id=e.nct_id
        INNER JOIN conditions c
            ON studies.nct_id=c.nct_id
        INNER JOIN browse_conditions bc
            ON studies.nct_id=bc.nct_id
        WHERE c.downcase_name='{}'
        LIMIT 5;
        """.format(condition)

r = conn.execute(q).fetchall()
col_names = conn.execute(q).keys()

pd.DataFrame(r, columns=col_names)

,nct_id,gender,minimum_age,maximum_age,healthy_volunteers,population,eligibility_criteria,brief_summary,condition
0,NCT02847767,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n All t...,\n The high dose per fraction (>10Gy/frac...,"carcinoma, hepatocellular"
1,NCT02847767,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n All t...,\n The high dose per fraction (>10Gy/frac...,"carcinoma, hepatocellular"
2,NCT02794337,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n - ...,\n Vast majority of patients with hepatoc...,"carcinoma, hepatocellular"
3,NCT02794337,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n - ...,\n Vast majority of patients with hepatoc...,"carcinoma, hepatocellular"
4,NCT01605734,All,18 Years,75 Years,No,,\n Inclusion Criteria:\n\n - ...,\n TACE is widely used in patients with u...,"carcinoma, hepatocellular"


In [6]:
# Identify total number of trials for all with condition
q = """SELECT count(DISTINCT studies.nct_id)
        FROM studies
        FULL OUTER JOIN brief_summaries bs
            ON studies.nct_id=bs.nct_id
        FULL OUTER JOIN eligibilities e
            ON studies.nct_id=e.nct_id
        INNER JOIN conditions c
            ON studies.nct_id=c.nct_id
        INNER JOIN browse_conditions bc
            ON studies.nct_id=bc.nct_id
        WHERE c.downcase_name='{}';
        """.format(condition)

r = conn.execute(q).fetchall()
r[0][0]


222

In [7]:
# Extract information from all HCC trials
# Perform selection of brief_summary
condition = "carcinoma, hepatocellular"

q = """SELECT studies.nct_id,
            e.gender,
            e.minimum_age,
            e.maximum_age,
            e.healthy_volunteers,
            e.population,
            e.criteria AS eligibility_criteria,
            bs.description AS brief_summary,
            c.downcase_name AS condition
        FROM studies
        FULL OUTER JOIN brief_summaries bs
            ON studies.nct_id=bs.nct_id
        FULL OUTER JOIN eligibilities e
            ON studies.nct_id=e.nct_id
        INNER JOIN conditions c
            ON studies.nct_id=c.nct_id
        INNER JOIN browse_conditions bc
            ON studies.nct_id=bc.nct_id
        WHERE c.downcase_name='{}';
        """.format(condition)

r = conn.execute(q).fetchall()
col_names = conn.execute(q).keys()

df = pd.DataFrame(r, columns=col_names)

# Drop duplicate rows
df.drop_duplicates(inplace=True)

df 

,nct_id,gender,minimum_age,maximum_age,healthy_volunteers,population,eligibility_criteria,brief_summary,condition
0,NCT02847767,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n All t...,\n The high dose per fraction (>10Gy/frac...,"carcinoma, hepatocellular"
2,NCT02794337,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n - ...,\n Vast majority of patients with hepatoc...,"carcinoma, hepatocellular"
4,NCT01605734,All,18 Years,75 Years,No,,\n Inclusion Criteria:\n\n - ...,\n TACE is widely used in patients with u...,"carcinoma, hepatocellular"
6,NCT01594125,All,20 Years,N/A,No,,\n Inclusion criteria:\n\n 1. ...,\n The aim of the study is to investigate...,"carcinoma, hepatocellular"
8,NCT01668134,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n - ...,\n This study is being done to determine ...,"carcinoma, hepatocellular"
11,NCT01556815,All,18 Years,80 Years,No,,\n Inclusion Criteria:\n\n - ...,\n Transarterial chemoembolization (TACE)...,"carcinoma, hepatocellular"
22,NCT01078415,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n - ...,\n The purpose of this study is to evalua...,"carcinoma, hepatocellular"
24,NCT02878109,All,18 Years,N/A,No,The target population is adults with a hepatoc...,\n Inclusion Criteria:\n\n - ...,\n This study is conducted to evaluate dy...,"carcinoma, hepatocellular"
26,NCT02847468,All,18 Years,99 Years,No,,\n Inclusion Criteria:\n\n - ...,\n Hepatocellular carcinoma (HCC) is the ...,"carcinoma, hepatocellular"
28,NCT02115373,All,18 Years,N/A,No,,\n Inclusion Criteria:\n\n - ...,"\n This is a Phase 1b/2, multicenter, sin...","carcinoma, hepatocellular"


## Process text

In [8]:
def clean_text(text):
    
    elements_to_remove = [':',
                           '-',
                           '\n',
                          '\t',
                          ]
    for element in elements_to_remove:
        text = text.replace(element, '')
    
    return text.lower()
    
df.eligibility_criteria.apply(clean_text)[0]

"        inclusion criteria        all the following criteria must be met          1. age > 18 years old          2. multiphase ct scan and/or mri of the liver within 8 weeks of radiation planning             demonstrating                 liver tumours < 5 cm                 no more than 2 discrete liver tumours                 normal liver > 700 cc          3. patients must have hcc diagnosed by either i) pathological confirmation, or ii)             intrahepatic vascular enhancement of the lesion demonstrated by at least two imaging             modalities, or iii) intrahepatic vascular enhancement of the lesion demonstrated by             one imaging modality if afp > 200 in the setting of liver cirrhosis or chronic             hepatitis b without cirrhosis (easl consensus guidelines [2])          4. liver hcc must be deemed unresectable as determined by an experienced hepatobiliary             surgeon, or the patient must be medically inoperable or refuse surgery,          5. patien

## Tokenize words and take frequencies using nltk

In [9]:
from nltk.tokenize import word_tokenize
from nltk import bigrams
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [10]:
def remove_stopwords(text, stopwords):
    return [word for word in text if word not in stopwords]

stop_words = set(stopwords.words('english'))

df.eligibility_criteria.apply(clean_text).apply(word_tokenize).apply(remove_stopwords, stopwords=stop_words)

0      [inclusion, criteria, following, criteria, mus...
2      [inclusion, criteria, diagnosis, hcc, ., tissu...
4      [inclusion, criteria, patients, newly, diagnos...
6      [inclusion, criteria, 1., histologically/cytol...
8      [inclusion, criteria, age, ≥, 18, karnofsky, p...
11     [inclusion, criteria, patients, newly, diagnos...
22     [inclusion, criteria, hcc, diagnosed, positive...
24     [inclusion, criteria, patients, least, 18, yea...
26     [inclusion, criteria, patient, older, 18, year...
28     [inclusion, criteria, histologically, confirme...
30     [inclusion, criteriaeach, patient, eligible, p...
31     [inclusion, criteria, hepatocellular, carcinom...
33     [inclusion, criteria, 1., hepatocellular, carc...
35     [inclusion, criteria, age, >, 18, years, hcc, ...
37     [inclusion, criteria, participants, must, conf...
39     [inclusion, criteria, clinical, diagnosis, hep...
41     [inclusion, criteria, patients, hepatocellular...
43     [inclusion, criteria, he

In [11]:
bigrams = df['eligibility_criteria'].apply(clean_text). \
    apply(word_tokenize).  \
    apply(remove_stopwords, stopwords=stop_words). \
    apply(lambda row: list(bigrams(row))). \
    apply(pd.Series).stack().reset_index(drop=True) # Turn series of lists into stacked series

In [ ]:
# Bigram frequencies
val_counts = bigrams.value_counts()[:100]

plt.figure(figsize=(12,6))
plt.bar(val_counts.index.map(str), val_counts.values);

In [15]:
bigrams.value_counts()[6:100]

(., patients)                 121
(4, weeks)                    120
(performance, status)         102
(6, months)                   102
(18, years)                    85
(., exclusion)                 78
(life, expectancy)             77
(criteria, 1.)                 77
((, ))                         75
(patients, must)               73
(3, months)                    69
(dose, study)                  67
(criteria, patients)           66
(first, dose)                  66
(days, prior)                  65
(within, 4)                    64
(least, one)                   62
(upper, limit)                 62
(limit, normal)                59
(written, informed)            59
(oncology, group)              59
(study, drug)                  58
(prior, study)                 58
(childpugh, class)             58
(eastern, cooperative)         57
(cooperative, oncology)        57
(weeks, prior)                 56
(portal, vein)                 54
(therapy, ,)                   53
(study, entry)

Analysis of the top most frequent bigrams from eligibilites of 222 HCC trials showed common features from these trials. These bigrams can now be used to identify general wording structures. For instance, lengths of time (eg, "2 weeks" or "4 weeks") or can be extracted from these

- 4 weeks, 2 weeks, 30 days, 
- performance status
- 6 months, 8 months
- 18 years
- within 4
- basal cell
- upper limit, limit normal
- life expectancy: life expectancy of at least 3 months
- chemotherapy
- prior studies
- portal vein
- patients must
- patient must
- with the last
- virus
- grade 2
- child pugh: Child-Pugh class A
- ECOG performance
- prior systemic
- nervous system
- locoregional therapy
- ECOG
- 

In [25]:
df.loc[df['eligibility_criteria'].str.contains('child'), 'eligibility_criteria'][22]

'\n        Inclusion Criteria:\n\n          -  HCC diagnosed by positive biopsy or non-invasive criteria,\n\n          -  not suitable for surgical resection or transplantation,\n\n          -  have at least one, but less than or equal to 3 tumors,\n\n          -  of the tumour(s) identified, each tumor must be ≤ 3 cm in diameter,\n\n          -  Child-Pugh class A,\n\n          -  Eastern Cooperative Oncology Group (ECOG) score of 0,\n\n          -  American Society of Anaesthesiologists (ASA) score ≤ 3,\n\n          -  a prothrombin time ratio > 50%,\n\n          -  platelet count > 50x109/L,\n\n          -  ability of patient to stop anticoagulant and anti-platelet therapy for seven days\n             prior to and seven days post NanoKnife procedure,\n\n          -  are able to comprehend and willing to sign the written informed consent form (ICF),\n\n          -  have a life expectancy of at least 3 months.\n\n        Exclusion Criteria:\n\n          -  eligible for surgical treatm